In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# model with single input

In [21]:
input1 = layers.Input(shape=(3, 4, 6))
added = layers.Add()([input1, tf.ones_like(input1)])
model = keras.models.Model(inputs=input1, outputs=added)

### show inference

In [23]:
model(tf.ones((1, 3, 4, 6)))

<tf.Tensor: shape=(1, 3, 4, 6), dtype=float32, numpy=
array([[[[2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.]],

        [[2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.]],

        [[2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.]]]], dtype=float32)>

In [45]:
model(np.array([[1,2], [3,4], [5,6]]))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[2., 3.],
       [4., 5.],
       [6., 7.]], dtype=float32)>

## save tflite model

In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp4m1v3r_y/assets


/home/garbade/.pyenv/versions/3.8.5/envs/py38_venv/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
INFO:tensorflow:Assets written to: /tmp/tmp4m1v3r_y/assets


In [25]:
with open("adder_model_single_input_3x4x6.tflite", "wb") as file:
    file.write(tflite_model)

## Example inference using tflite model

In [54]:
# TFLite quantized inference example
#
# Based on:
# https://www.tensorflow.org/lite/performance/post_training_integer_quant
# https://www.tensorflow.org/lite/api_docs/java/org/tensorflow/lite/Tensor.QuantizationParams

import numpy as np
import tensorflow as tf

# Location of tflite model file (float32 or int8 quantized)
# model_path = "adder_model_single_input.tflite"
model_path = "adder_model_single_input_2x3.tflite"
# model_path = "signn_static.tflite"

# Processed features (copy from Edge Impulse project)
# features = [1.0, 2.0]
x = np.arange(6)
features = x.reshape((2, 3))
  
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=model_path)

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Allocate tensors
interpreter.allocate_tensors()

# Print the input and output details of the model
print()
print("Input details:")
print(input_details)
print()
print("Output details:")
print(output_details)
print()

# Convert features to NumPy array
np_features = np.array(features)

# If the expected input type is int8 (quantized model), rescale data
input_type = input_details[0]['dtype']
    
# Convert features to NumPy array of expected type
np_features = np_features.astype(input_type)

# Add dimension to input sample (TFLite model expects (# samples, data))
np_features = np.expand_dims(np_features, axis=0)

# Create input tensor out of raw features
interpreter.set_tensor(input_details[0]['index'], np_features)

# Run inference
interpreter.invoke()

# output_details[0]['index'] = the index which provides the input
output = interpreter.get_tensor(output_details[0]['index'])

# If the output type is int8 (quantized model), rescale data
output_type = output_details[0]['dtype']

# Print the results of inference
print("Inference output:", output)


Input details:
[{'name': 'input_7', 'index': 0, 'shape': array([1, 2, 3], dtype=int32), 'shape_signature': array([-1,  2,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

Output details:
[{'name': 'Identity', 'index': 4, 'shape': array([1, 2, 3], dtype=int32), 'shape_signature': array([-1,  2,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

Inference output: [[[1. 2. 3.]
  [4. 5. 6.]]]


In [55]:
input_type

numpy.float32

In [52]:
functions = []
for i in range(10):
    functions.append(lambda i: i)

for i, f in enumerate(functions):
    print(f(i))

0
1
2
3
4
5
6
7
8
9
